## Description
### This Jupyter Notebook is used for obtaining descriptors values from virtual candidates after descriptor selection

#### Workflow:
1. The training dataset in the form of a .csv file is read into a dataframe, followed by generating a Pearson Correlation Table to identify collinearity within the set of descriptors. 
2. Descriptors which are highly collinear are removed.
3. The training data is then standardized. Two features with the highest Pearson Coefficient with the target variable are chosen for classifier training. With two features, we are able to visualize the classifier with a decision boundary on a 2D map.
4. Best hyperparameters are obtained for each type of classifiers.
5. Decision boundaries are created and visualized.
6. Further training is conducted by using charge of ligand as an additional descriptor.

In [30]:
#Import libraries
import re #Import RegEx
import os
import math
import ast
import numpy as np
import pandas as pd
from pathlib import Path

In [31]:
# Assign directory names
directory = os.getcwd()
nbo_directory = directory + '\\nbo'
opt_directory = directory + '\\opt'
steric_map_dir = directory + '\\steric_maps_nocoord_candidate'
parent_dir = os.path.dirname(directory)
directory

'C:\\Users\\George\\Desktop\\Research_UCLA\\CIC\\Computational Work\\Project-3-2\\intermediates\\virtual_candidates'

In [32]:
# Aims
# Extract descriptor from output file of spe.
# Extract geometric descriptors (bond angles, bond lengths) and electronic descriptors (nbo charge, HOMO-LUMO gap)
# Input needed: No. for the donor atoms (A and B) as well as metal atom (Pd)

# Bond distance: given cartesian coordinates, find distance between atoms
# Bond angle: given three atoms, find angle


#Input excel table containing metal, atom_a and atom_b name
df = pd.read_csv('input_candidate.csv')     


xyz_match = ['X           Y           Z']
LUMO_match = ['Alpha virt. eigenvalues']
HOMO_match = ['Alpha  occ. eigenvalues']
nbo_match = ['Natural Population Analysis']
spe_match = ['SCF Done']


#List for constructing dataframe
df_name = []
df_metal = []
df_atom_a = []
df_atom_b = []
df_angle = []
df_metal_atoma_distance = []
df_metal_atomb_distance = []
df_metal_nbo = []
df_a_nbo = []
df_b_nbo = []
df_HOMO_LUMO = []
df_category = []
df_ligand = []
df_AB_NBO = []
df_MAMB_distance = []
df_spe = []
df_natoms = []

for subdir,dirs,files in os.walk(nbo_directory):     # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem           # Extract filename from the end of path and return as a string
            
         
            for x in df['Filename']:                      # For each element in the "Filename" column of the input file
                if x == name:
                    match_file = df[df['Filename'] == name] 
#                     print(match_file)

            metal = match_file.iloc[0,1]                    # Extract metal, atom_a and atom_b numbers from input files
            atom_a = match_file.iloc[0,2]
            atom_b = match_file.iloc[0,3]
            
            
#             print(name, 'Metal: ',metal, 'Atom_A: ', atom_a, 'Atom_B: ', atom_b)

            mylines = []

            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # myfile = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines list.


                # Find XYZ Coordinates

                for line in mylines:
                    if 'NAtoms' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])

                for line in mylines:
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line

                                                                                    # For loop for generating the XYZ coordinates
                            count = 0
                            xyz = []
                            line_number = mylines.index(line) + 2
                            while count < natoms:
                                count = count + 1
                                xyz.append(mylines[line_number])
                                line_number = line_number + 1
                            
#                             # Convert into xyz file 
                            

#                             with open('xyz.txt', 'w') as filehandle:                   # Save xyz coordinates (list) into a txt file
#                                 for listitem in xyz:
#                                     filehandle.write(listitem)

                x_coord = []
                y_coord = []
                z_coord = []
                atom_symbol =[]
                element = {"1":'H', "6":"C", "7": "N", "8":"O", "9":"F", "15":"P", "16":"S", "17":"Cl", "26":"Fe", "35":"Br", "46":"Pd"}
                
                # Generate XYZ file in .txt form, then find xyz coordinates for metal, atom_a and atom_b
                for line in xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])
                    element_number = int(number_list[1])
                    atom_x = float(number_list[3])
                    atom_y = float(number_list[4])
                    atom_z = float(number_list[5])
                    
                    # Make xyz coord into .txt file for percent buried volume
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    name_xyz = name + '_xyz.txt'            
                    xyz_df = pd.DataFrame([atom_symbol,x_coord,y_coord,z_coord])
                    xyz_df = xyz_df.transpose()
#                     xyz_df.to_csv(name_xyz, header=False, index=False, sep = " ")

                    # Identify metal, atom_a and atom_b numbers and obtain their respective x,y,z coords
                    if atom_number == metal:
                        metal_x = atom_x
                        metal_y = atom_y
                        metal_z = atom_z
                        if element_number != 46:
                            print('Wrong metal atom number assignment ',filename) # Checking if metal atom was assigned correctly
                    elif atom_number == atom_a:
                        a_x = atom_x
                        a_y = atom_y
                        a_z = atom_z
                    elif atom_number == atom_b:
                        b_x = atom_x
                        b_y = atom_y
                        b_z = atom_z
                        
                
                    

                #Extract metal atom distance
                
                metal_atoma_distance = math.sqrt((metal_x - a_x) ** 2 + (metal_y - a_y) ** 2 + (metal_z - a_z) ** 2)    
                metal_atomb_distance = math.sqrt((metal_x - b_x) ** 2 + (metal_y - b_y) ** 2 + (metal_z - b_z) ** 2)

                metal_atoma_vector = [a_x - metal_x, a_y - metal_y, a_z - metal_z]
                metal_atomb_vector = [b_x - metal_x, b_y - metal_y, b_z - metal_z]

                dot_pdt = np.dot(metal_atoma_vector,metal_atomb_vector)
                    
                #Extract A-M-B angle by using arcos rule
                angle = int(math.degrees(math.acos (dot_pdt / ( metal_atoma_distance * metal_atomb_distance ))))         

#                 print('A-M-B Angle: ', angle)
#                 print('Metal-AtomA Distance: ', metal_atoma_distance)
#                 print('Metal-AtomB Distance: ', metal_atomb_distance)



                # Find HOMO - LUMO energy gap

                LUMO = []
                for phrase in LUMO_match:
                    for line in mylines:
                        if phrase in line:
                            LUMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',LUMO[0])            # get value from line containing LUMO
                LUMO_energy = float(number_list[0])                         # get first value from the line, which represents the LUMO

                HOMO = []
                for phrase in HOMO_match:
                    for line in mylines:
                        if phrase in line:
                            HOMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',HOMO[-1])            # get value from line containing LUMO
                HOMO_energy = float(number_list[-1])                        # get lsat value from line, which represents HOMO

                HOMO_LUMO = LUMO_energy - HOMO_energy                       # HOMO LUMO gap, in A.U.
#                 print('HOMO-LUMO Gap: ', HOMO_LUMO)

                # Find NBO charge

                for phrase in nbo_match:                                       # iterate through each phrases
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line

                            count = 0
                            nbo_xyz = []
                            line_number = mylines.index(line) + 6
                            while count < natoms:
                                count = count + 1
                                nbo_xyz.append(mylines[line_number])
                                line_number = line_number + 1

                            with open('nbo_xyz.txt', 'w') as filehandle:                   # Save nbo_xyz  into a txt file
                                for listitem in nbo_xyz:
                                    filehandle.write(listitem)

                                                                                            # Extract NBO charge for metal, atom A and atom B
                for line in nbo_xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])

                    if atom_number == metal:
                        metal_nbo = float(number_list[1])

                    elif atom_number == atom_a:
                        a_nbo = float(number_list[1])

                    elif atom_number == atom_b:
                        b_nbo = float(number_list[1])
                        
                # Find SPE 
                
                for phrase in spe_match:
                    for line in mylines:
                        if phrase in line:
                            spe = line.replace("SCF Done:  E(RM06) =  " , "")
                            l = len(spe)
                            spe = float(spe[:l-27])
                df_spe.append(spe)
                            
                # Naming category
                


                if 'nocoord' in name:
                    category = 'nocoord'
                    df_category.append('nocoord')
                    name = name.replace('-spe-nocoord',"")
                elif 'cocoord' in name:
                    category = 'cocoord'
                    df_category.append('cocoord')
                    name = name.replace('-spe-cocoord',"")
                elif 'etcoord' in name:
                    category = 'etcoord'
                    df_category.append('etcoord') 
                    name = name.replace('-spe-etcoord',"")
                else:
                    print(name, 'error, no category assigned')
                 
                print(name, category, natoms)
                df_name.append(name)
                df_metal.append(metal)
                df_atom_a.append(atom_a)
                df_atom_b.append(atom_b)
                df_angle.append(angle)
                df_metal_atoma_distance.append(metal_atoma_distance)
                df_metal_atomb_distance.append(metal_atomb_distance)
                df_metal_nbo.append(metal_nbo)
                df_a_nbo.append(a_nbo)
                df_b_nbo.append(b_nbo)
                df_HOMO_LUMO.append(HOMO_LUMO)
                df_AB_NBO.append(a_nbo-b_nbo)
                df_MAMB_distance.append(metal_atoma_distance-metal_atomb_distance)
                df_natoms.append(natoms)
#                 df_ligand.append(name[:-12])
                
                
data = {'Name': df_name,
        'Category' : df_category,
       'Metal': df_metal, 
       'Atom_A' : df_atom_a, 
       'Atom_B' : df_atom_b, 
       'A-M-B Angle' : df_angle, 
       'M-A Distance' : df_metal_atoma_distance, 
       'M-B Distance' : df_metal_atomb_distance, 
       'Metal_NBO' : df_metal_nbo, 
       'A_NBO' : df_a_nbo, 
       'B_NBO' : df_b_nbo, 
       'HOMO_LUMO' : df_HOMO_LUMO,
       'A-B_NBO' : df_AB_NBO,
       'MA_MB_Difference' : df_MAMB_distance,
       'SPE': df_spe}       
                
# for key, value in data.items():
#     #print value
#     print(key, len([item for item in value if item]))
    
    
# print ('Name: ', len(df_name),
#        'Category: ', len(df_category),
#        'Metal: ', len(df_metal))
    




descriptor_data = pd.DataFrame(data)
descriptor_data = descriptor_data.sort_values('Category')
print(descriptor_data)
descriptor_data.to_csv("output_nbo.csv") 


checking_data = {'Name': df_name,
                 'Category' : df_category,
                 'NAtoms': df_natoms} 

checking_df = pd.DataFrame(checking_data)
checking_df.to_csv("checking_df.csv") 

c-n-1 etcoord 63
c-n-2 etcoord 61
c-n-3 etcoord 70
c-o-1 etcoord 59
c-o-2 etcoord 86
c-o-3 etcoord 92
c-o-4 etcoord 68
c-o-5 etcoord 54
c-o-6 etcoord 68
c-o-9 etcoord 94
c-p-1 etcoord 66
c-s-1 etcoord 48
p-o-10 etcoord 62
p-o-11 etcoord 64
p-o-12 etcoord 65
p-o-13 etcoord 84
p-o-15 etcoord 92
p-o-16 etcoord 94
p-o-17 etcoord 80
p-o-18 etcoord 80
p-o-19 etcoord 98
p-o-20 etcoord 80
p-o-21 etcoord 74
p-o-22 etcoord 78
p-o-23 etcoord 81
p-o-24 etcoord 79
p-o-25 etcoord 73
p-o-26 etcoord 77
p-o-27 etcoord 59
p-o-28 etcoord 91
p-o-29 etcoord 81
p-o-30 etcoord 87
p-o-31 etcoord 91
p-o-32 etcoord 97
p-o-34 etcoord 70
p-o-35 etcoord 71
p-o-36 etcoord 100
p-o-37 etcoord 70
p-o-38 etcoord 94
p-o-39 etcoord 100
p-o-40 etcoord 96
p-o-43 etcoord 100
p-o-44 etcoord 96
Wrong metal atom number assignment  C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\nbo\etcoord\p-o-48-spe-etcoord.out
p-o-48 etcoord 73
p-o-8 etcoord 86
p-o-9 etcoord 76
c-n-1 

In [33]:
# Aims
# Extract gibbs free energy from output file of opt.

import re #Import RegEx
import os
import math
import numpy as np
import pandas as pd
import re #Import RegEx
import os
from pathlib import Path

df_filename = []
df_gibbs = []
df_category = []
df_ligand = []

match_list = ['Thermal correction to Gibbs Free Energy']

rootdir = os.getcwd()                                       # Get current working directory

count = 0

for subdir,dirs,files in os.walk(opt_directory):            # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file

            mylines = []

            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # contents = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines.
                    
                for line in mylines:
                    if 'NAtoms' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])


                for phrase in match_list:                                       # iterate through each phrases
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line
                            name = Path(filename).stem
                            line = line.replace("Thermal correction to Gibbs Free Energy=         " , "")
                            l = len(line)
#                             line = line[:l-27]
                            gfe_corr = float(line)
                            df_gibbs.append(gfe_corr)
                            

                            if 'nocoord' in name:
                                category = 'nocoord'
                                df_category.append('nocoord')
                                name = name.replace('-opt-nocoord',"")
                            elif 'cocoord' in name:
                                category = 'cocoord'
                                df_category.append('cocoord')
                                name = name.replace('-opt-cocoord',"")
                            elif 'etcoord' in name:
                                category = 'etcoord'
                                df_category.append('etcoord') 
                                name = name.replace('-opt-etcoord',"")
                            else:
                                print('error, no category assigned')
                            df_filename.append(name)
                            print(name, category, natoms)
                            
                            
                            if name in checking_df['Name'].tolist():
                                to_check_df = checking_df[(checking_df['Name'] == name) & (checking_df['Category'] == category)]
                                
                                if len(to_check_df) == 0:
                                    print('opt name not in spe list')
                                
                                elif natoms != to_check_df['NAtoms'].tolist()[0]:
                                    print('opt and spe file doesnt match')
                                
                            else:
                                print('opt name not in spe list')
 

                            
delta_g_data = {'Name': df_filename, 'Category': df_category, 'Gibbs Free Energy Correction' : df_gibbs}      

descriptor_data = pd.DataFrame(delta_g_data)
descriptor_data.to_csv("output_deltaG.csv")                              
                            
                            # number_list = re.findall('-?\d*\.?\d+',line)            # get value
                            # for x in number_list:                                   # print value
                            #     print(phrase,': ', float(x))



c-n-1 etcoord 63
c-n-2 etcoord 61
c-n-3 etcoord 70
c-o-1 etcoord 59
c-o-2 etcoord 86
c-o-3 etcoord 92
c-o-4 etcoord 68
c-o-6 etcoord 68
c-o-9 etcoord 94
c-p-1 etcoord 66
c-s-1 etcoord 48
p-o-10 etcoord 62
p-o-11 etcoord 64
p-o-12 etcoord 65
p-o-13 etcoord 84
p-o-15 etcoord 92
p-o-16 etcoord 94
p-o-17 etcoord 80
p-o-18 etcoord 80
p-o-19 etcoord 98
p-o-20 etcoord 80
p-o-21 etcoord 74
p-o-22 etcoord 78
p-o-23 etcoord 81
p-o-24 etcoord 79
p-o-25 etcoord 73
p-o-26 etcoord 77
p-o-27 etcoord 59
p-o-28 etcoord 91
p-o-29 etcoord 81
p-o-30 etcoord 87
p-o-31 etcoord 91
p-o-32 etcoord 97
p-o-34 etcoord 70
p-o-35 etcoord 71
p-o-36 etcoord 100
p-o-37 etcoord 70
p-o-38 etcoord 94
p-o-39 etcoord 100
p-o-40 etcoord 96
p-o-43 etcoord 100
p-o-44 etcoord 96
p-o-48 etcoord 73
p-o-8 etcoord 86
p-o-9 etcoord 76
c-n-1 nocoord 57
c-n-2 nocoord 55
c-n-3 nocoord 64
c-o-1 nocoord 53
c-o-2 nocoord 80
c-o-3 nocoord 86
c-o-4 nocoord 62
c-o-5 nocoord 56
c-o-6 nocoord 62
c-o-7 nocoord 53
c-o-8 nocoord 72
c-p-1 nocoord

In [34]:
# Merge nbo and gibbs free energy dataset
# Calculate delta G difference between etcoord and nocoord


df_opt = pd.read_csv('output_deltaG.csv')
df_opt = df_opt.drop('Unnamed: 0',axis=1)

df_nbo = pd.read_csv('output_nbo.csv')
df_nbo = df_nbo.drop('Unnamed: 0',axis=1)

outer_merge = pd.merge(df_nbo,df_opt, how="outer", on=["Name", "Category"])
print(outer_merge.shape)
outer_merge.to_csv("output_merge.csv") 

(109, 16)


In [35]:
category_unique = outer_merge["Category"].unique()
for x in list(category_unique):
    df = outer_merge[outer_merge.Category == x]
    df.to_csv("df_" + x + ".csv") 

In [36]:
# Obtain relevant descriptors from nocoord-geometry only 

df_nocoord = pd.read_csv('df_nocoord.csv')
df_nocoord = df_nocoord.drop('Unnamed: 0',axis=1)

df_nocoord.rename(columns = {'SPE':'SPE_nocoord', 
                             'Gibbs Free Energy Correction':'Gibbs_Free_Energy_Correction_nocoord'}, inplace = True)
df_nocoord = df_nocoord.dropna()
columns_to_drop = ['Category', 
                   'Metal',
                   'Atom_A',
                   'Atom_B',
                   'SPE_nocoord',
                   'Gibbs_Free_Energy_Correction_nocoord']
df_nocoord = df_nocoord.drop(columns_to_drop,axis=1)



print(df_nocoord)

      Name  A-M-B Angle  M-A Distance  M-B Distance  Metal_NBO    A_NBO  \
0    c-n-1         84.0      2.097219      2.050283    0.33113  0.24361   
1    c-n-2         90.0      2.139646      2.157782    0.37354  0.21914   
2    c-n-3         84.0      2.104480      2.206040    0.38173  0.24008   
3    c-o-1         87.0      2.080869      2.027901    0.39489  0.32829   
4    c-o-2         81.0      2.117490      2.134671    0.46711  0.23290   
..     ...          ...           ...           ...        ...      ...   
58  p-o-43         91.0      2.435518      2.074618    0.41279  0.99537   
59  p-o-44         92.0      2.433349      2.068452    0.37789  1.00058   
60  p-o-45         94.0      2.452727      2.092091    0.41164  0.99748   
61   p-o-8         89.0      2.443324      2.079455    0.38382  0.99616   
62   p-o-9         94.0      2.453381      2.093052    0.37722  0.99476   

      B_NBO  HOMO_LUMO  A-B_NBO  MA_MB_Difference  
0  -0.86651    0.15855  1.11012          0.0469

In [37]:
# Merge with steric descriptors
df_buriedvol = pd.read_csv('percent_buried_volume_candidate.csv')
new_merge_2 = pd.merge(df_nocoord,df_buriedvol, how="outer", on=["Name"])
new_merge_2=new_merge_2.dropna()                


## Determine %Filled for virtual candidates and its weighted %Filled

In [38]:
#Set up new dataframe

dict = {'Name':[],
        'Axis':[],
        'Boundary':[],
        'Threshold':[],
        'Unfilled':[],
        'Filled':[],
        '%Filled':[],
       }

new_df = pd.DataFrame(dict)

for filename in os.listdir(steric_map_dir):
    csv_name = filename.replace(".csv","")
    name = csv_name.replace("-nocoord-map","")   #obtain catalyst name as a variable
    filepath = os.path.join(steric_map_dir,filename)
    print(name,filepath)
    dataset = pd.read_csv(filepath,header=None)
    dataset.columns = ["X-coord", "Y-coord", "Z"]

    # Set up datasets
    axis_list = ["X-coord",'Y-coord']
    min_list = [-1.00, -2.00, -3.00]
    max_list = [1.00, 2.00, 3.00]

    # Define upper and lower boundaries within the dataset
    for axis in axis_list:
        for min_boundary, max_boundary in zip(min_list,max_list):
            df_min = dataset[dataset[axis] >= min_boundary]
            df_minmax = df_min[df_min[axis] <= max_boundary]

    # Identify %Filled for each Z threshold based on voxel value

            threshold_list = list(np.arange(0,2,0.25))  # Define Z thresholds with 0.25 intervals
            boundary = str(min_boundary)+'-'+str(max_boundary)

            for threshold in threshold_list:
                df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
                df_minmax.loc[df_minmax['Z'] > threshold, 'Filled_Unfilled'] = 'Filled'


                if df_minmax.Filled_Unfilled.value_counts().Unfilled == len(df_minmax.index):  # If all voxel is unfilled
                    filled = 0
                    unfilled = df_minmax.Filled_Unfilled.value_counts().Unfilled
                    filled_percent = 0

                else:
                    unfilled, filled = df_minmax['Filled_Unfilled'].value_counts()  # If some voxel is filled
                    filled_percent = 100 * filled/(unfilled+filled)

                new_row = {'Name':name,
                           'Axis':axis,
                           'Boundary':boundary,
                           'Threshold':threshold, 
                           'Unfilled':unfilled, 
                           'Filled':filled, 
                           '%Filled':filled_percent}
                new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
                # new_df = new_df.append(new_row, ignore_index=True)

    #Add inversed dataset
    for axis in axis_list:
        for min_boundary, max_boundary in zip(min_list,max_list):
            df_overmax = dataset[dataset[axis] >= max_boundary]                                #Inversed boundaries
            df_lowermin = dataset[dataset[axis] <= min_boundary]
            df_combined = pd.concat([df_overmax,df_lowermin])


           # Identify %Filled for each Z threshold

            threshold_list = list(np.arange(0,2,0.25))
            boundary = '-3.45 - '+str(min_boundary)+' and '+str(max_boundary) + ' - 3.55'

            for threshold in threshold_list:
                df_combined.loc[df_combined['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
                df_combined.loc[df_combined['Z'] > threshold, 'Filled_Unfilled'] = 'Filled'



                if df_combined.Filled_Unfilled.value_counts().Unfilled == len(df_combined.index):
                    filled = 0
                    unfilled = df_combined.Filled_Unfilled.value_counts().Unfilled
                    filled_percent = 0

                else:
                    unfilled, filled = df_combined['Filled_Unfilled'].value_counts()
                    filled_percent = 100 * filled/(unfilled+filled)

                new_row = {'Name':name,
                           'Axis':axis,
                           'Boundary':boundary,
                           'Threshold':threshold, 
                           'Unfilled':unfilled, 
                           'Filled':filled, 
                           '%Filled':filled_percent}
                new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
                # new_df = new_df.append(new_row, ignore_index=True)

              
    
new_df = new_df.sort_values(by =['Axis', 'Boundary', 'Threshold'])

new_df.to_csv('steric_descriptors-virtual-candidate.csv')


c-n-1 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\steric_maps_nocoord_candidate\c-n-1-nocoord-map.csv


C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy 

c-n-2 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\steric_maps_nocoord_candidate\c-n-2-nocoord-map.csv


C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy 

c-n-3 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\steric_maps_nocoord_candidate\c-n-3-nocoord-map.csv


...

C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy 

p-o-8 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\steric_maps_nocoord_candidate\p-o-8-nocoord-map.csv


C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy 

p-o-9 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\virtual_candidates\steric_maps_nocoord_candidate\p-o-9-nocoord-map.csv


C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_56976\3270091328.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [39]:
# Obtaining coefficient for each descriptor

group_df = new_df.groupby(['Axis','Boundary','Threshold'])
# Obtain coefficient dataframe from training dataset
file_location = parent_dir + '\\training_set\\coefficient_df.csv'
coefficient_df = pd.read_csv(file_location)    


# for group_name, group_data in group_df:

new_df2 = pd.merge(new_df, coefficient_df, on=['Axis','Boundary','Threshold'], how="inner")



#Determine average weighted percentage filled for each structure
new_df2['Weighted_percent_filled'] = new_df2['%Filled'] * new_df2['Coefficient']

new_df2.to_csv('steric_descriptors-virtual-candidate-coefficient.csv')

group_df = new_df2.groupby(['Name'])
name_list = []
average_weighted_percent_filled = []
for group_name, group_data in group_df:
    weighted_values = group_data["Weighted_percent_filled"]  
    name_list.append(group_name[0])
    average = (sum(weighted_values)/len(weighted_values))
    average_weighted_percent_filled.append(average)
#     print (group_name, average)

print(name_list)

avg_weighted_percent_filled_tuples = list(zip(name_list, average_weighted_percent_filled))
avg_weighted_percent_filled_df = pd.DataFrame(avg_weighted_percent_filled_tuples, columns=['Name','Weighted %Filled'])

print(avg_weighted_percent_filled_df)




#     print(group_df.get_group(key)) 



['c-n-1', 'c-n-2', 'c-n-3', 'c-o-1', 'c-o-2', 'c-o-3', 'c-o-4', 'c-o-5', 'c-o-6', 'c-o-7', 'c-o-8', 'c-p-1', 'c-s-1', 'n-o-1', 'n-o-2', 'n-o-3', 'n-o-4', 'n-o-5', 'n-o-6', 'p-n-1', 'p-n-2', 'p-n-3', 'p-n-4', 'p-o-10', 'p-o-11', 'p-o-12', 'p-o-13', 'p-o-14', 'p-o-15', 'p-o-16', 'p-o-17', 'p-o-18', 'p-o-19', 'p-o-20', 'p-o-21', 'p-o-22', 'p-o-23', 'p-o-24', 'p-o-25', 'p-o-26', 'p-o-27', 'p-o-28', 'p-o-29', 'p-o-30', 'p-o-31', 'p-o-32', 'p-o-33', 'p-o-34', 'p-o-35', 'p-o-36', 'p-o-37', 'p-o-38', 'p-o-39', 'p-o-40', 'p-o-41', 'p-o-42', 'p-o-43', 'p-o-44', 'p-o-45', 'p-o-46', 'p-o-47', 'p-o-48', 'p-o-8', 'p-o-9']
      Name  Weighted %Filled
0    c-n-1          1.505013
1    c-n-2          1.384708
2    c-n-3          1.270216
3    c-o-1          1.125194
4    c-o-2          1.470405
..     ...               ...
59  p-o-46          2.213221
60  p-o-47          1.492040
61  p-o-48          0.398922
62   p-o-8          1.502584
63   p-o-9          0.577652

[64 rows x 2 columns]


In [40]:
x_max_steric_txt_filepath = parent_dir + '\\training_set\\x_max_steric.txt'
y_max_steric_txt_filepath = parent_dir + '\\training_set\\y_max_steric.txt'
file_path = 'path/to/file.txt'  # Replace with the relative file path

with open(x_max_steric_txt_filepath, 'r') as file:
    x_max_steric = file.read()
    x_max_steric_tuple = ast.literal_eval(x_max_steric)
    
with open(y_max_steric_txt_filepath, 'r') as file:
    y_max_steric = file.read()
    y_max_steric_tuple = ast.literal_eval(y_max_steric)

print(x_max_steric_tuple)
print(y_max_steric_tuple)

('X-coord', '-3.45 - -3.0 and 3.0 - 3.55', 0.0)
('Y-coord', '-1.0-1.0', 0.25)


In [41]:
avg_weighted_percent_filled_df

,Name,Weighted %Filled
0,c-n-1,1.505013
1,c-n-2,1.384708
2,c-n-3,1.270216
3,c-o-1,1.125194
4,c-o-2,1.470405
...,...,...
59,p-o-46,2.213221
60,p-o-47,1.492040
61,p-o-48,0.398922
62,p-o-8,1.502584


In [42]:
group_df = new_df.groupby(['Axis', 'Boundary','Threshold'])

# Obtain respective %Filled_x and %Filled_y from combinations of boundary/z-threshold obtained from training algorithm




x_max_steric = group_df.get_group(x_max_steric_tuple)
y_max_steric = group_df.get_group(y_max_steric_tuple)
    
x_max_steric = x_max_steric.drop(['Axis','Boundary','Threshold','Unfilled','Filled'],axis=1)
x_max_steric = x_max_steric.rename(columns={"%Filled": "%Filled_x"})
y_max_steric = y_max_steric.drop(['Axis','Boundary','Threshold','Unfilled','Filled'],axis=1)
y_max_steric = y_max_steric.rename(columns={"%Filled": "%Filled_y"})
new_merge_3 = pd.merge(pd.merge(pd.merge(new_merge_2,x_max_steric, how="inner", on=["Name"])
                                ,y_max_steric, how="inner", on=["Name"])
                                ,avg_weighted_percent_filled_df, how="inner", on=["Name"])

new_merge_3.to_csv("all_descriptor_data_virtual.csv")  
